In [8]:
##################Import Modules#######################
import random
import numpy as np
import math
import pandas as pd
import tensorflow as tf
tf.set_random_seed(777) 


##################Define Functions#####################
def five_fold_name(data,i):
    test_names = data[data['index']==i+1]
    
    
def five_fold(data, i):
    test_data = data[data['index']==i+1]
    train_data = data[(data['index']<i+1) | (data['index']>i+1)]
    print(len(test_data), len(train_data))
    
    return train_data , test_data

def set_train_three_layer(repeat, nodes, learning_rate):
    batch_size = 1000
    tf.reset_default_graph()
    keep_prob = tf.placeholder(tf.float32)
    
    X = tf.placeholder(tf.float32, [None, cnt_train])
    Y = tf.placeholder(tf.float32, [None, 2])

    W1 = tf.get_variable( shape= [cnt_train, nodes[0]], name='weight1' , initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([nodes[0]]), name='bias1')
    layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)

    W2 = tf.get_variable(shape =[nodes[0], nodes[1]], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([nodes[1]]), name='bias2')
    layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
    layer2 = tf.nn.dropout(layer2 , keep_prob=keep_prob)

    W3 = tf.get_variable(shape= [nodes[1], nodes[2]], name='weight3',initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([nodes[2]]), name='bias3')
    layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
    layer3 = tf.nn.dropout(layer3, keep_prob=keep_prob)

    W4 = tf.get_variable(shape=[nodes[2], 2], name='weight4',initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([2]), name='bias4')
    hypothesis = tf.matmul(layer3, W4) + b4



    # cost/loss function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



    # Accuracy computation
    # True if hypothesis>0.5 else False

    predicted = tf.argmax(hypothesis,1)
    correct_prediction = tf.equal(predicted,tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
    with tf.Session() as sess:
        # Initialize TensorFlow variables
        sess.run(tf.global_variables_initializer())

        for step in range(repeat):
            avg_cost = 0
            total_num = int(len(train_x)/batch_size)

            for i in range(total_num):
                batch_x = train_x[i*batch_size:(i+1)*batch_size]
                batch_y = train_y[i*batch_size:(i+1)*batch_size]
                sess.run( train , feed_dict={X: batch_x, Y: batch_y , keep_prob : 0.7})

            if step == repeat-1:
                ####Train Accuracy report####
                train_h, train_p, train_a = sess.run([hypothesis, predicted, accuracy],feed_dict={X: train_x, Y: train_y, keep_prob :0.7})
                print("\nTrain Accuracy: ", train_a)
            if step % 20 == 0 :
                train_h,c, train_p,train_a = sess.run([hypothesis, cost ,predicted, accuracy],feed_dict={X: train_x, Y: train_y, keep_prob :0.7})
                print("\nCurrent Accuracy : ", train_a , "cost : ", c , "Current Step : ", step)
                if train_a > 0.97 :
                    break
        test_h, test_p, test_a = sess.run([hypothesis, predicted, accuracy],feed_dict={X: test_x, Y: test_y, keep_prob :1.0})
        print("\nTest Accuracy: ", test_a)

    return train_p ,train_h, test_p,test_h 

##################READ DATA############################
datafilename = "/Users/taewan/Desktop/Ahn/FinalData_GSM_gene_index_result.csv"
data = pd.read_csv(datafilename)
repeat, layer, node , learning_rate, gene = 100, 3,'1500 1500 1500' ,0.01 , 60
output_directory = '/home/tjahn/Git/Data/output/'

for j in range(5):
    #####Five fold#####
    train_data, test_data = five_fold(data, j)
    train_GSM = train_data.iloc[:,0]
    test_GSM = test_data.iloc[:,0]
    print(train_GSM, test_GSM)
    #####Train Data Set#####
    train_x = train_data.iloc[:,1:-2]
    train_x = train_x.as_matrix()
    train_y = train_data.iloc[:,-2].as_matrix()
    train_y = train_y.flatten()
    train_y = pd.get_dummies(train_y)

    #####Test Data Set#####
    test_x = test_data.iloc[:,1:-2]
    test_x = test_x.as_matrix()
    test_y = test_data.iloc[:,-2].as_matrix()
    test_y = test_y.flatten()
    test_y = pd.get_dummies(test_y)

    cnt_train = len(train_x[1, :])
    train_p, train_h , test_p ,test_h = (set_train_three_layer(repeat, nodes, learning_rate))
    train_result = pd.concat([train_GSM, train_p], axis=1)
    train_result = pd.concat([train_result, train_h], axis =1 )
    test_result = pd.concat([test_GSM, test_p], axis=1)
    test_result = pd.concat([test_result, test_h], axis=1)
    
    result_train_filename = "result_file_train" + str(j) +".csv"
    train_result.to_csv(output_directory+result_train_filename , sep= ',')
    result_test_filename = "result_file_test" + str(j) +".csv"
    test_result.to_csv(output_directory+result_test_filename , sep= ',')
    ###train h를 file로
    ###test h를 file로 

2432 9481
0        GSM253210
1        GSM253216
2        GSM256745
3        GSM256746
5        GSM256748
7        GSM256750
8        GSM257187
9        GSM257190
10       GSM257248
11       GSM257249
13       GSM257526
14       GSM257527
15       GSM257528
16       GSM257660
17       GSM257661
18       GSM257471
19       GSM257472
20       GSM259831
21       GSM259832
24       GSM259835
25       GSM259836
26       GSM260250
27       GSM260251
28       GSM260252
29       GSM260253
30       GSM260254
31       GSM260255
32       GSM260256
35       GSM260533
37       GSM260535
           ...    
11879    GSM251078
11880    GSM251079
11881    GSM251098
11882    GSM251099
11883    GSM251103
11884    GSM251108
11885    GSM251101
11886    GSM251105
11887    GSM251110
11888    GSM251111
11889    GSM251114
11890    GSM251126
11891    GSM251129
11893    GSM251194
11894    GSM251195
11895    GSM251196
11896    GSM251197
11897    GSM251198
11898    GSM251199
11900    GSM251207
11902    GSM251209
11